# Context Window: Why Chatbots Forget
Every model has a context window limit - the maximum number of tokens it can process in a single request.

This includes input + output tokens combined.

Let's see what happens when you exceed it!

In [2]:
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')
API_KEY = os.environ["GEMINI_API_KEY"]
client = genai.Client(api_key=API_KEY)

In [3]:
# Create a VERY long message to hit the limit
long_story = "Once upon a time, there was a programmer who loved Python. " * 100000

print(f"📏 Story length: {len(long_story):,} characters")
print(f"📏 Estimated tokens: ~{len(long_story) // 4:,}\n")

try:
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=long_story,
        config={"max_output_tokens": 100}
    )
    print("✅ Request succeeded!")
    print(f"📊 Tokens used: {response.usage_metadata.total_token_count:,}")
    print(f"\n💡 Gemini 2.5 Flash has a HUGE context window (~1M tokens)")
    print(f"   So this request fits comfortably!")
except Exception as e:
    print(f"❌ ERROR: {e}")
    print("\n💡 This is what happens when you exceed the context window!")

📏 Story length: 5,900,000 characters
📏 Estimated tokens: ~1,475,000

❌ ERROR: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 250000, model: gemini-2.5-flash-lite\nPlease retry in 19.061927089s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerMo

💡 This is what happens when you exceed the context window!
# What is Context Window?

## Context Window = Maximum tokens in a single request

Different models have different limits:

* Gemini 2.5 Flash: ~1 million tokens
* GPT-4: ~128K tokens
* Claude 4: ~200K tokens

# Context Window in Conversations
In chat applications, the context window includes:

* All previous messages (entire conversation history)
* Current message
* Response

As conversations grow, tokens accumulate!

In [7]:
messages = []

def chat(user_message):
    """Send message and track tokens"""
    messages.append(
        types.Content(role="user", parts=[types.Part(text=user_message)])
    )
    
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=messages
    )
    
    messages.append(
        types.Content(role="model", parts=[types.Part(text=response.text)])
    )
    
    # Show token usage
    total_tokens = response.usage_metadata.total_token_count
    print(f"🤖: {response.text}")
    print(f"📊 Total tokens used: {total_tokens} (includes ALL {len(messages)} messages)\n")
    
    return response.text

# Start conversation
print("👤: Hi! My name is Atharva")
chat("Hi! My name is Atharva")

print("👤: I'm 25 years old")
chat("I'm 25 years old")

print("👤: I love Python programming")
chat("I love Python programming")

print("👤: I work as a software engineer")
chat("I work as a software engineer")

print("👤: What's my name?")
chat("What's my name?")

👤: Hi! My name is Atharva
🤖: Hi Atharva! It's nice to meet you. How can I help you today?
📊 Total tokens used: 29 (includes ALL 2 messages)

👤: I'm 25 years old
🤖: Thanks for letting me know, Atharva! That's a great age. What can I do for you today?
📊 Total tokens used: 64 (includes ALL 4 messages)

👤: I love Python programming
🤖: That's fantastic, Atharva! Python is a wonderfully versatile and popular programming language. I'm a big fan of it myself.

What do you enjoy most about Python? Are you working on any specific projects, or are you interested in learning something new about it? I'm here to chat about anything related to Python!
📊 Total tokens used: 139 (includes ALL 6 messages)

👤: I work as a software engineer
🤖: That's excellent, Atharva! Working as a software engineer is a very rewarding and dynamic field. It's great that you're applying your passion for Python in your career.

What kind of software do you typically work on? Are you in web development, data science, backend

'Your name is **Atharva**.'

In [6]:
MAX_MESSAGES = 6  # Keep only last 6 messages (3 exchanges)

def chat_with_limit(user_message):
    """Chat with context window management"""
    messages.append(
        types.Content(role="user", parts=[types.Part(text=user_message)])
    )
    
    # Remove old messages if too many
    if len(messages) > MAX_MESSAGES:
        removed = messages.pop(0)  # Remove oldest
        print(f"🗑️  Removed old message: {removed.parts[0].text[:50]}...\n")
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=messages
    )
    
    messages.append(
        types.Content(role="model", parts=[types.Part(text=response.text)])
    )
    
    print(f"🤖: {response.text}")
    print(f"📊 Messages in memory: {len(messages)}\n")
    
    return response.text

# Reset and try again
messages = []

print("👤: My favorite color is blue")
chat_with_limit("My favorite color is blue")

print("👤: I have a dog named Max")
chat_with_limit("I have a dog named Max")

print("👤: I live in Mumbai")
chat_with_limit("I live in Mumbai")

print("👤: I enjoy hiking")
chat_with_limit("I enjoy hiking")

# This will forget the first message!
print("👤: What's my favorite color?")
chat_with_limit("What's my favorite color?")

👤: My favorite color is blue
🤖: That's a wonderful choice! Blue is such a versatile and beautiful color, often associated with peace, the sky, and the ocean.

Do you have a favorite shade of blue, like navy, sky blue, sapphire, or perhaps a vibrant turquoise?
📊 Messages in memory: 2

👤: I have a dog named Max
🤖: Oh, how wonderful! Having a dog named Max sounds absolutely lovely! Max is such a classic and beloved dog name.

Dogs bring so much joy and companionship to our lives. I'd love to hear more about him if you'd like to share!

*   What kind of dog is Max?
*   Is he a playful pup or more of a relaxed companion?
*   How old is he?
*   What are some of his favorite things to do?

Tell me all about your good boy, Max!
📊 Messages in memory: 4

👤: I live in Mumbai
🤖: Mumbai! What an incredible and vibrant city. I've heard so much about its bustling energy, incredible food, rich history, and beautiful coastal views. It sounds like a fascinating place to live!

Do you enjoy living there?

"That's a fun question! As an AI, I don't actually know your personal preferences, including your favorite color. I don't have access to information about you beyond what you tell me in our conversation.\n\nSo, you'll have to tell me! What *is* your favorite color? I'm curious!"

## Key Takeaways
Context Window = Maximum tokens in a single request (input + output)

In conversations, tokens grow with each message

Must remove old messages when approaching limit

This causes "forgetting" - AI loses old context

Exceeding limit = Error - Request will fail

## Real-World Solutions
Summarize old messages instead of removing
Store important info separately (database)
Prioritize recent messages
Use RAG (Retrieval Augmented Generation) for long-term memory
Monitor token usage and manage proactively